In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import sys
import pandas as pd
import h5py

In [48]:
rootDir = '/d/DATD/datd/pRF_orig/'
# List all subjects in the root directory
subjects = os.listdir(rootDir)
subjects = [x for x in subjects if not x.startswith('.')]
# Remove HSL1110_RF1_raw, and oldTemp and XIZ1020_bummer
subjects = [x for x in subjects if not x.startswith('HSL1110') and not x.startswith('oldTemp') and not x.startswith('XIZ1020')]
TaskTypeSTIMF = {}
TaskTypeFRAMES = {}

for subject in subjects:
    print(subject)
    # Check number of folders that start with RF
    rfFolders = [x for x in os.listdir(os.path.join(rootDir, subject)) if x.startswith('RF')]
    # Check if subject_RFx_vista exists inside the RF folder
    for rfFolder in rfFolders:
        mrVistaPath = os.path.join(rootDir, subject, rfFolder, subject + '_' + rfFolder + '_vista')
        if os.path.exists(mrVistaPath):
            # Check if folder called Stimuli exists
            stimuliPath = os.path.join(mrVistaPath, 'Stimuli')
            if os.path.exists(os.path.join(stimuliPath)):
                # List all files in the stimuli folder
                stimFiles = os.listdir(stimuliPath)
                # Check if any of files have the string '1300ms'
                stimFiles1300 = [x for x in stimFiles if '1300ms' in x]
                stimFiles1200 = [x for x in stimFiles if '1200ms' in x]
                if len(stimFiles1300) > 0:
                    TaskTypeSTIMF[subject] = 'RSVP'
                if len(stimFiles1200) > 0:
                    TaskTypeSTIMF[subject] = 'RDK'
                if len(stimFiles1300) > 0 and len(stimFiles1200) > 0:
                    print('Both RSVP and RDK found in subject: ' + subject)
                if len(stimFiles1300) == 0 and len(stimFiles1200) == 0:
                    print('No RSVP or RDK found in subject: ' + subject)

                # Find the file mrSESSION.mat in mrVistaPath
                mrSessionPath = os.path.join(mrVistaPath, 'mrSESSION.mat')

                # Copy the file to '/Users/mrugank/Desktop/tempHolder
                os.system('cp ' + mrSessionPath + ' /Users/mrugank/Desktop/tempHolder')
                # Load the file
                mat = loadmat('/Users/mrugank/Desktop/tempHolder/mrSESSION.mat')
                # Delete the file
                os.system('rm /Users/mrugank/Desktop/tempHolder/mrSESSION.mat')

                nFrames = mat['dataTYPES'][0][0][1][0][0][1][0][0]
                if nFrames == 304:
                    TaskTypeFRAMES[subject] = 'RDK'
                elif nFrames == 201 or nFrames == 211:
                    TaskTypeFRAMES[subject] = 'RSVP'
                else:
                    print('Unknown task type for subject: ' + subject + ' with nFrames: ' + str(nFrames))

AB
AC
AF
AJF
AL0825
AL_old
AM
AS
AY
BB
CC
No RSVP or RDK found in subject: CC
Unknown task type for subject: CC with nFrames: 462
CZ
DP
DS
DT
EK
EK7T
EMS0201
EY
FH
GH
HHL
HL
HT
JC
JH
JK
JS
JSH
JW
K4S0421
KD
KF
KN
LG
LL
LR
LS
MA
MAH
MB
MD
ME
MG
MH
ML
MP
MR
MSR
MTR
NA
NM
NZ
PL
QL
QY
RAC
RC
RH
RW
SF
SH
SHL
SK
SL
SM
SN
SSK
SSL
SW
SX
TA
TZ
WP
XL
YIL
YK
YL
YS
YUL
YX
YZ
ZD
ZDU
ZHL
ZL
ZZ


In [56]:
for subject in subjects:
    print(subject)
    # Check number of folders that start with RF
    rfFolders = [x for x in os.listdir(os.path.join(rootDir, subject)) if x.startswith(subject)]
    # Ignore folders that end with anat
    rfFolders = [x for x in rfFolders if not x.endswith('anat')]
    if len(rfFolders) > 0:
        print(subject, rfFolders)

AB
AC
AF
AJF
AL0825
AL_old
AM
AS
AY
BB
CC
CC ['CC2']
CZ
DP
DS
DT
EK
EK7T
EMS0201
EY
FH
GH
HHL
HL
HL ['HL_2mm']
HT
JC
JH
JK
JS
JSH
JW
K4S0421
KD
KF
KN
KN ['KN_9thrun']
LG
LG ['LG_01_procformat', 'LG_RF1_raw']
LL
LR
LS
MA
MAH
MB
MD
ME
ME ['ME_08-09-2022_RSVP_pRF.zip', 'MEold']
MG
MH
ML
MP
MP ['MPanat_backup']
MR
MSR
MTR
NA
NM
NZ
PL
QL
QY
QY ['QY_RF1_25mm']
RAC
RC
RC ['RC_old']
RH
RW
SF
SH
SH ['SH_25mm']
SHL
SHL ['SHL_01_procformat', 'SHL_02_preprocRF', 'SHL_03_pRFmodelfit_20mm', 'SHL_RF1_raw', 'SHL_zl_try']
SK
SL
SM
SM ['SM_2mm']
SN
SSK
SSL
SW
SX
TA
TA ['TA_old']
TZ
WP
XL
YIL
YK
YL
YS
YUL
YUL ['YUL_RF1_raw', 'YUL_RF2_raw']
YX
YZ
YZ ['YZ_RF1']
ZD
ZDU
ZHL
ZL
ZZ


In [51]:
TaskTypeSTIMF_df = pd.DataFrame(TaskTypeSTIMF.items(), columns=['Subject', 'TaskType'])
TaskTypeSTIMF_df.to_csv('TaskTypeSTIMF.csv', index=False)

In [52]:
TaskTypeFRAMES_df = pd.DataFrame(TaskTypeFRAMES.items(), columns=['Subject', 'TaskType'])
TaskTypeFRAMES_df.to_csv('TaskTypeFRAMES.csv', index=False)

In [53]:
# Check if any subject in TaskTypeSTIMF is not in TaskTypeFRAMES
for subject in TaskTypeSTIMF.keys():
    tType = TaskTypeSTIMF[subject]
    tTypeFrames = TaskTypeFRAMES[subject]
    if tType != tTypeFrames:
        print('Subject: ' + subject + ' has different task types: ' + tType + ' and ' + tTypeFrames)

Subject: AC has different task types: RSVP and RDK
Subject: AY has different task types: RSVP and RDK
Subject: BB has different task types: RSVP and RDK
Subject: DS has different task types: RSVP and RDK
Subject: GH has different task types: RSVP and RDK
Subject: JK has different task types: RSVP and RDK
Subject: KN has different task types: RSVP and RDK
Subject: LS has different task types: RSVP and RDK
Subject: MH has different task types: RSVP and RDK
Subject: MSR has different task types: RSVP and RDK
Subject: SF has different task types: RSVP and RDK
Subject: SH has different task types: RSVP and RDK
Subject: XL has different task types: RSVP and RDK
Subject: YK has different task types: RSVP and RDK
Subject: YS has different task types: RSVP and RDK
Subject: YZ has different task types: RSVP and RDK
